# 浜名湖4月8日 9R 結果

In [5]:
sample_html = \
    'http://boatrace.jp/owpc/pc/race/raceresult?rno=9&jcd=06&hd=20200410'

In [6]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re

## htmlをsoupに取り込む

In [7]:
target_url = 'http://boatrace.jp/owpc/pc/race/raceresult?rno=9&jcd=06&hd=20200410'
__html_content = urlopen(target_url).read()
__soup = BeautifulSoup(__html_content, 'html.parser')

### 結果表を抜く

In [7]:
__target_table_selector = \
    'body > main > div > div > div > '\
    'div.contentsFrame1_inner > div.grid.is-type2.h-clear.h-mt10 > '\
    'div:nth-child(1) > div > table'
__target_table_html = __soup.select_one(__target_table_selector)
player_res_html_list = __target_table_html.select('tbody')

In [46]:
waku_dict = {}
for rank_p_html in player_res_html_list:
    rank, waku, name, time = \
        list(map(lambda x: x.text, rank_p_html.select('td')))
    # rankはF,L欠などが存在するためエラーハンドルがいる
    try:
        rank = int(rank)
    except ValueError:
        rank = -1
#     time = time.replace('"', '.')
    waku = int(waku)
    name = name.replace('\n', '').replace('\u3000', '').replace(' ', '')
    no, name = name.split('\r')
    no = int(no)

    __content_dict = {
        'rank': rank,
        'name': name,
        'time': time
    }

    waku_dict[waku] = __content_dict

### 結果STテーブルを抜く

In [24]:
from getdata import CommonMethods4Official
cmo = CommonMethods4Official()
__target_table_selector = \
    'body > main > div > div > div > '\
    'div.contentsFrame1_inner > '\
    'div.grid.is-type2.h-clear.h-mt10 > '\
    'div:nth-child(2) > div > table'
st_list = cmo._getSTtable2tuple(__soup, __target_table_selector, 1)

In [7]:
goutei_list = list(
            map(lambda x: int(x.select('div > span')[0].text),
                st_list))

In [8]:
goutei_list

[1, 3, 6, 2, 4, 5]

In [30]:
__tenji_C_idx = goutei_list.index(6)
tenji_C = __tenji_C_idx + 1

In [31]:
# 展示ST抜き出し
tenji_st_time_list = list(
    map(lambda x: x.select('div > span')[2].text,
        st_list))

In [32]:
tenji_ST = __tenji_st_time_list[__tenji_C_idx]

In [33]:
tenji_ST = cmo.rmletter2float(tenji_ST.replace('F', '-'))

### 水面気象情報

In [12]:
from getdata import CommonMethods4Official
cmo = CommonMethods4Official()
table_selector = \
    'body > main > div > div > div > '\
    'div.contentsFrame1_inner > div:nth-child(5) > '\
    'div:nth-child(2) > div.grid.is-type6.h-clear > '\
    'div:nth-child(1) > div > div.weather1_body.is-type1__3rdadd'
__target_table_html = __soup.select_one(table_selector)
condinfo_html_list = __target_table_html.select('div')
assert len(condinfo_html_list) == 12, \
    f"lengh is not 12:{len(condinfo_html_list)}"
# 気温は2番目のdiv
__tmp_info_html = condinfo_html_list[1]
# spanで情報がとれる (1番目： '気温', 2番目: 数字℃)
__tmp_info = __tmp_info_html.select('span')
temp = __tmp_info[1].text
temp = cmo.rmletter2float(temp)
# 天気は2番目のdiv
__weather_info_html = condinfo_html_list[2]
# spanのなか（1個しかない)
weather = __weather_info_html.select_one('span').text
weather = weather.replace('\n', '')\
                 .replace('\r', '')\
                 .replace(' ','')
# 風速は5番目のdiv
wind_v = cmo.choose_2nd_span(condinfo_html_list[4])
wind_v: int = cmo.rmletter2int(wind_v)

# 水温は8番目のdiv
w_temp = cmo.choose_2nd_span(condinfo_html_list[7])
w_temp = cmo.rmletter2float(w_temp)

# 波高は10番目のdiv
wave = cmo.choose_2nd_span(condinfo_html_list[9])
wave = cmo.rmletter2int(wave)

# 風向きは7番目のdiv
# 画像のみの情報なので，16方位の数字（画像の名前）を抜く
# p中のクラス名2番目にある
wind_dr = condinfo_html_list[6].select_one('p')['class'][1]
wind_dr = cmo.rmletter2int(wind_dr)

content_dict = {
    'temp': temp,
    'weather': weather,
    'wind_v': wind_v,
    'w_temp': w_temp,
    'wave': wave,
    'wind_dr': wind_dr
}

In [13]:
content_dict

{'temp': 16.0,
 'weather': '晴',
 'wind_v': 6,
 'w_temp': 16.0,
 'wave': 4,
 'wind_dr': 3}

### 返還挺の確認

In [19]:
sample1 = 'http://boatrace.jp/owpc/pc/race/raceresult?rno=9&jcd=06&hd=20200410'
sample2 = 'http://boatrace.jp/owpc/pc/race/raceresult?rno=2&jcd=12&hd=20170906'
sample3 = 'http://boatrace.jp/owpc/pc/race/raceresult?rno=10&jcd=06&hd=20200410'
target_url = sample3
__html_content = urlopen(target_url).read()
__soup = BeautifulSoup(__html_content, 'html.parser')

table_selector = \
'body > main > div > div > div > div.contentsFrame1_inner > '\
'div:nth-child(5) > div:nth-child(2) > div.grid.is-type6.h-clear > '\
'div:nth-child(2) > div:nth-child(1) > table > tbody > tr > td > '\
'div > div span.numberSet1_number'
henkantei_html_list = __soup.select(table_selector)

In [20]:
# 返還艇をint型に直す，変なやつはNoneでハンドル（あんまりないけど）
def teistr2str(tei_str):
    tei = re.search(r'[1-6]', tei_str)
    if tei is not None:
        return str(tei.group(0))
    else:
        return None

# 返還艇があればリスト長が1以上になる
if len(henkantei_html_list) != 0:
    henkantei_list = list(map(
        lambda x: teistr2str(x.text), henkantei_html_list))
    henkantei_list = [n for n in henkantei_list if n is not None]
    is_henkan = True
else:
    henkantei_list = []
    is_henkan = False
henkantei_str = ','.join(henkantei_list)

In [21]:
henkantei_str

''

### 決まり手取得

In [117]:
table_selector = \
    'body > main > div > div > div > '\
    'div.contentsFrame1_inner > div:nth-child(5) > '\
    'div:nth-child(2) > div.grid.is-type6.h-clear > '\
    'div:nth-child(2) > div:nth-child(2) > table > tbody > tr > td'

In [118]:
kimarite = __soup.select_one(table_selector).text

### 備考

In [126]:
table_selector = \
    'body > main > div > div > div > '\
    'div.contentsFrame1_inner > div:nth-child(5) > '\
    'div:nth-child(2) > div.table1 > table > tbody > tr > td'

In [127]:
biko = __soup.select_one(table_selector).text

In [130]:
biko.replace('\r', '')\
    .replace('\n', '')\
    .replace(' ', '')

'【返還艇あり】'

In [137]:
import sys
import logging
logger = logging.getLogger()
handler = logging.StreamHandler(sys.stdout)
handler.setLevel(logging.DEBUG)
logger.addHandler(handler)
logger.setLevel(logging.DEBUG)

from getdata import OfficialResults
ors = OfficialResults(
    3, 6, 20200408
)

In [138]:
ors.getplayerresult2dict(1)

{'rank': 2,
 'name': '鈴木裕隆',
 'no': 4231,
 'racetime': 111.5,
 'course': 1,
 'st_time': 0.16}

### 払い戻し，人気

In [25]:
table_selector = \
    'body > main > div > div > div > '\
    'div.contentsFrame1_inner > div:nth-child(5) > '\
    'div:nth-child(1) > div > table'
pay_pop_tb = __soup.select_one(table_selector)
pay_pop_tb_list = pay_pop_tb.select('tbody')

In [67]:
payout_3t = pay_pop_tb_list[5].select_one('span.is-payout1').text
payout_3t = int(payout_3t.replace('¥', '').replace(',', ''))
popular_3t = pay_pop_tb_list[5].select_one('tr:nth-child(1) > td:nth-child(4)').text
popular_3t = cmo._rmletter2int(popular_3t)

In [68]:
print(payout_3t, popular_3t)

410 None


In [62]:
type(pay_pop_tb)

bs4.element.Tag

In [4]:
','.join([None])

TypeError: sequence item 0: expected str instance, NoneType found

In [1]:
from getdata import OfficialResults
ors = OfficialResults(1,21,20200512)
ors.getraceresult2dict()

{'temp': 21.0,
 'weather': '曇り',
 'wind_v': 5,
 'w_temp': 21.0,
 'wave': 5,
 'wind_dr': 3,
 'henkantei_list': '',
 'is_henkan': False,
 'kimarite': '逃げ',
 'biko': '\xa0',
 'payout_3tan': 4000,
 'popular_3tan': 14,
 'payout_3fuku': 1540,
 'popular_3fuku': 7,
 'payout_2tan': 1120,
 'popular_2tan': 5,
 'payout_2fuku': 690,
 'popular_2fuku': 3,
 'payout_1tan': 270}

In [1]:
from dt2sql import ResultData2sql
rd2s = ResultData2sql()
rd2s.insert2table(20200512,21,1)

{'temp': 21.0, 'weather': '曇り', 'wind_v': 5, 'w_temp': 21.0, 'wave': 5, 'wind_dr': 3, 'henkantei_list': '', 'is_henkan': False, 'kimarite': '逃げ', 'biko': '\xa0', 'payout_3tan': 4000, 'popular_3tan': 14, 'payout_3fuku': 1540, 'popular_3fuku': 7, 'payout_2tan': 1120, 'popular_2tan': 5, 'payout_2fuku': 690, 'popular_2fuku': 3, 'payout_1tan': 270}


In [2]:
type(ResultData2sql)

abc.ABCMeta

In [3]:
type(rd2s.insert2table)

method